In [1]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import csv
import os.path

In [12]:
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [2]:
SlopesList = {}
linkIds = []
SlopeReader = csv.reader(open('data/Partition6467LinkData.csv'))
count=0
for row in SlopeReader:
    SlopesList[row[0]]=[]
    slopes = []
    altitude = row[14].split('|')[0].split('/')[2]
    if altitude:
        SlopesList[row[0]].append(float(altitude))
    else:
        SlopesList[row[0]].append(None)
    for slope in row[16].split('|'):
        if slope:
            dist, degree = slope.split('/')
            slopes.append((float(dist), float(degree)))
    SlopesList[row[0]].append(slopes)
    linkIds.append(row[0])
    count += 1
    if count%10000==0:
        print(count)
print('Slope Loading done!')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
Slope Loading done!


In [3]:
probeReader = csv.reader(open("data\Partition6467MatchedPoints.csv"), delimiter=",")
matchedP = []
#probeIds = []
count=0
temp=(int(3496),float(51),float(9),float(200))#the initial

#ID = 3496 #the first ID
for row in probeReader:
    point = (int(row[8]),float(row[3]),float(row[4]),float(row[5]))#linkID,latitude,longitude,altitude
    esse = (row[8], float(row[5]), float(row[10]), float(row[11])) 
    count+=1
    matchedP.append(esse)
    #print(type(point))
    #only if ID is the same, the slope can be computed
    if count%50000==0:
        print(count)

print('Probe data loaded')

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
Probe data loaded


In [4]:
slopecal = {}
for matchedP in matchedP:
    LinkId = matchedP[0]
    matcheddis = matchedP[2]
    matchedAlt = matchedP[1]
    link = SlopesList[LinkId]
    refAlt = link[0]
    
    if refAlt is None or matcheddis == 0:
        degree = -1
    else:
        diffAlt = matchedAlt - refAlt
        degree = math.degrees(math.atan(diffAlt/matcheddis))
    slopecal.setdefault(LinkId, []).append((matcheddis, degree))
    
    count += 1
    if count%50000==0:
        print(count)

print("Slope calculated ")

3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000
5050000
5100000
5150000
5200000
5250000
5300000
5350000
5400000
5450000
5500000
5550000
5600000
5650000
5700000
5750000
5800000
5850000
5900000
5950000
6000000
6050000
6100000
6150000
6200000
6250000
6300000
6350000
6400000
6450000
6500000
6550000
6600000
6650000
6700000
Slope calculated 


In [9]:
if not os.path.isfile('slope.csv') :
    with open("data/Partition6467LinkData.csv", "r") as pp, open("slope.csv", "w") as out:
        for link,row in zip(linkIds,pp):
            line = row.strip()
            if link in slopecal:
                linkSlope = slopecal[link]
                linkSlope = sorted(linkSlope, key=lambda x:x[0])
                line += "," + '|'.join(['/'.join(map(str,slope)) for slope in linkSlope])

            out.write(line + "\n")



In [26]:
# borrowed from github, using multiple standard to verify the function
goodcount2 = 0
goodcount5 = 0
badsample=0
count=0

with open("slope.csv", "r") as pp:
    ppreader = csv.reader(pp)
    for row in ppreader:
        if len(row)>17 and row[16] and row[17]:
            probeSlopes = [tuple(map(float, x.split('/'))) for x in row[17].split('|')]
            linkSlopes = [tuple(map(float, x.split('/'))) for x in row[16].split('|')]
            availSlopes=[]
            linkSlopes = sorted(linkSlopes, key=lambda x:x[0])
            for slope in probeSlopes:
                dist = slope[0]
                degree = slope[1]
                if degree == -1:
                    break 
                for i in range(0, len(linkSlopes)):
                    diff=[]
                    if linkSlopes[i][0] >= dist:
                        break
                prevslope = 0
                nextslope = 0
                if i - 1 >= 0 :
                    prevslope = linkSlopes[i - 1][1]
                if i >= 0 :
                    nextslope = linkSlopes[i][1]
                linkslope = (nextslope + prevslope) / 2
                diff = np.absolute(degree - linkslope)
                availSlopes.append((dist, degree, diff, linkslope))

            for slope in availSlopes:
                count+=1
                diff = slope[2]
                if diff < 2:
                    goodcount2+=1
                if diff < 5:
                    goodcount5+=1
                if diff > 60:
                    badsample+=1

print ("%d out of  %d slopes matched , %f%%, has less than 2 degrees error" % (goodcount2, count, goodcount2*100/float(count)))
print ("%d out of  %d slopes matched , %f%%, has less than 5 degrees error" % (goodcount5, count, goodcount5*100/float(count)))
print ("%d out of  %d slopes totally different , %f%%, has more than 60 degrees error" % (badsample, count, badsample*100/float(count)))

999127 out of  1775307 slopes matched , 56.279111%, has less than 2 degrees error
1309653 out of  1775307 slopes matched , 73.770508%, has less than 5 degrees error
35462 out of  1775307 slopes totally different , 1.997514%, has more than 60 degrees error
